In [ ]:
import os
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import glob
import os
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.cluster import KMeans
import tensorflow as tf
from sklearn.svm import LinearSVC
from skimage import feature
import numpy as np
from skimage import feature
import numpy as np
numPoints=24
radius=6
eps=1e-7

In [ ]:
cancer_image=[]
labels=[]
n=100
for cancer_dir_path in glob.glob("/home/rishotics/sawon/data/*"):
    cancer_label=cancer_dir_path.split("/")[-1]
    
    path="/home/rishotics/sawon/data/"+cancer_label+"/40X/*"
    c=0
    for cancer_dir_path_res in glob.glob(path):
        
        if(c==n):
            break
        else:
            image=cv.imread(cancer_dir_path_res,-1)
            image = cv.resize(image, (460,700))
            image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
            labels.append(cancer_label)
            cancer_image.append(image)
            c=c+1
            
        
        
cancer_image=np.array(cancer_image)
labels=np.array(labels)
cancer_to_dict={i:k for k,i in enumerate(np.unique(labels))}
dict_to_id={i:k for k,i in cancer_to_dict.items()}
label_id=np.array([cancer_to_dict[x] for x in labels])
        
    

In [ ]:
data = []
labels = []

def LBP_features(image):
    #gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    
    
    
    lbp = feature.local_binary_pattern(image, numPoints,radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, numPoints + 3),
    range=(0, numPoints + 2))
 
		# normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
   # hist = hist.describe()
    return (pd.Series(hist).describe())

In [ ]:
j=0
X=np.zeros((cancer_image.shape[0],8))
for i in cancer_image:
    
    
    
    lbp_features=LBP_features(i)
    
    X[j,:]=lbp_features
    
    j=j+1
X.shape
X    